In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymysql

In [3]:
# Functions

# reverse the data of argument
def reverseData(data):
    data = data[::-1]
    return data

def getRound(data, decimals = 3):
    return np.around(data, decimals)

## given two curve a, b, then return the inversection point of two curves
def FindTheBuyPoint(a, b):
    buyPoint = np.argwhere(np.diff(np.sign(a - b)) > 0).reshape(-1) + 0
    return buyPoint

def FindTheSellPoint(a, b):
    sellPoint = np.argwhere(np.diff(np.sign(a - b)) < 0).reshape(-1) + 0
    return sellPoint

def GetDataFromDB(ip, userName, psw, tableName, sql):
    try:
        db = pymysql.connect(ip, userName, psw, tableName)
    except:
        print ("Error: unable to connect to DB")
        
    #create a cursor
    cursor = db.cursor()
    results = []
    try:
        cursor.execute(sql)
        results = cursor.fetchall()
    except:
        print ("Error: unable to fetch data from DB")
    
    return results
    db.close()

def SMA(data, period):
    if (type(data) == pd.core.series.Series):
        return data.rolling(window = period).mean()
    else:
        print ("Error: Wrong input, SMA(pandas.core.series.Series, integer)")
        
def EMA(data, period):
    if (type(data) == pd.core.series.Series):
        return data.ewm(span = period, adjust = False).mean()
    else:
        print ("Error: Wrong input, EMA(pandas.core.series.Series, integer)")
        
def RSI(data, period, Uprevious = None, Dprevious = None):
    if (type(data) == pd.core.series.Series):
        delta = data.diff()
        up, down = delta.copy(), delta.copy()
        up[up < 0] = 0
        down[down > 0] = 0
        down = abs(down)
        Up = SMA(up, period)
        Up = Up.fillna(0)
        if(Uprevious == None):
            for i in range(period,len(Up)):
                Up[i] = getRound((Up[i - 1] * (period - 1) + up[i]) / period)
        else:
            Up[period - 1] = getRound((Up[i - 1] * (period - 1) + up[i]) / period)
            
        Down = SMA(down, period)
        Down = Down.fillna(0)
        if(Uprevious == None):
            for i in range(period,len(Down)):
                Down[i] = getRound((Down[i - 1] * (period - 1) + down[i]) / period)
        else:
            Down[period - 1] = getRound((Dprevious * (period - 1) + down[i]) / period)

        rsi = 100 * (Up / (Up + Down))
        rsi = rsi.fillna(0)
        return getRound(rsi, 2), Up, Down
    else:
        print ("Error: Wrong input, RSI(pandas.core.series.Series, integer)")
        
def MACD(data, period = []):
    if (type(data) == pd.core.frame.DataFrame and len(period) == 3):   
        di = (data['High'] + data['Low'] + 2.0 * data['Close']) / 4.0
        ema12 = SMA(di, period[0])
        ema12 = ema12.fillna(0)
        for i in range(period[0] + 1, len(ema12)):
            ema12[i] = (ema12[i - 1] * (period[0] - 1) + di[i] * 2.0) / (period[0] + 1)
    
        ema26 = SMA(di, period[1])
        ema26 = ema26.fillna(0)
        for i in range(period[1] + 1, len(ema26)):
            ema26[i] = (ema26[i - 1] * (period[1] - 1) + di[i] * 2.0) / (period[1] + 1)
    
        dif = ema12 - ema26

        dem = SMA(dif, period[2])
        dem = dem.fillna(0)
        for i in range(period[2] + 1, len(dem)):
            dem[i] = (dem[i - 1] * (period[2] - 1) + dif[i] * 2.0) / (period[2] + 1)
        return {'dif':dif, 'dem':dem}
    else:
        if(type(data) == pd.core.frame.DataFrame):
            print ("Error: Wrong input, MACD(pandas.core.frame.DataFrame, list of integer)")
        elif(len(period) == 3):
            print ("Error: number of content in list do not equal to 3")
            
def KD(data, result = {}):
    close = data['Close'].copy()
    for i in range(0,9):
        close[i] = 0

    data['RSV'] = (( data['Close'] - data['Low'].rolling(window = 9).min()) / (data['High'].rolling(window = 9).max() - data['Low'].rolling(window = 9).min()))
    data['RSV'] = data['RSV'].fillna(0)
    if(not result):
        result = {
            'K9':[0],
            'D9' :[0]
        }
    #calculate everyday's KD
    for i in range(1, len(data.index)):
        K9_value = (1.0/3.0) * data['RSV'][i] + (2.0 / 3.0) * result['K9'][i - 1]
        result['K9'].append(getRound(K9_value, 5))
        D9_value = (2.0/3.0) * result['D9'][i - 1] + (1.0 / 3.0) * result['K9'][i]
        result['D9'].append(getRound(D9_value, 5))
        
    return result

In [5]:
# define a class - indicators, which contains all indicators that calculate by the data
class Indicators:
    data = pd.DataFrame()
    EMA = pd.DataFrame()
    SMA = pd.DataFrame()
    RSI = pd.DataFrame()
    KD = pd.DataFrame()
    MACD = pd.DataFrame()
    
    def __init__(self, inputData):
        self.data = inputData
        self.SMA = self.setSMA([5, 10, 20, 60, 120, 240])
        self.EMA = self.setEMA([5, 20, 100])
        self.RSI = self.setRSI(14)
        self.KD = self.setKD()
        self.MACD = self.setMACD([12, 26, 9])
    
    def setSMA(self, periods = []):
        smaResults = []
        for i in range(len(periods)):
            if (type(periods[i]) != int):
                print ("Error: inputData is not a list of integer")
                return
            else:
                smaResults.append(SMA(self.data['Close'], periods[i]))
                
        return pd.DataFrame(dict((("SMA" + str(periods[i])), smaResults[i]) for i in range(0, len(smaResults))))
    
    def getSMA(self):
        return self.SMA
    
    def setEMA(self, periods = []):
        emaResults = []
        for i in range(len(periods)):
            if (type(periods[i]) != int):
                print ("Error: inputData is not a list of integer")
                return
            else:
                emaResults.append(EMA(self.data['Close'], periods[i]))
                
        return pd.DataFrame(dict((("EMA" + str(periods[i])), emaResults[i]) for i in range(0, len(emaResults))))
    
    def getEMA(self):
        return self.EMA
    
    def setRSI(self, period):
        if (type(period) != int):
            print ("Error: inputData is not an integer")
            return
        
        rsiResults = RSI(self.data['Close'], period)
        return pd.DataFrame({"RSI": list(rsiResults)})
    
    def getRSI(self):
        return self.RSI
    
    def setKD(self):
        kdResults = KD(self.data)
        return pd.DataFrame(kdResults)
    
    def getKD(self):
        return self.KD
    
    def setMACD(self, periods = []):
        macdResults = []
        for i in range(len(periods)):
            if (type(periods[i]) != int):
                print ("Error: inputData is not a list of integer")
                return
        macdResults = MACD(self.data, periods)
        return pd.DataFrame(macdResults)
    
    def getMACD(self):
        return self.MACD
    
    def getLong_ShortPoints(self, indicator):
        if (type(indicator) != str):
            print ("Error: the type of argument is wrong : it should be str type")
            return
        longPoints = []
        shortPoints = []
        if(indicator.lower() == 'sma'):
            longPoints_5_20 = FindTheBuyPoint(self.SMA['SMA5'].head(365), self.SMA['SMA20'].head(365))
            longPoints_5_100 = FindTheBuyPoint(self.SMA['SMA5'].head(365), self.SMA['SMA100'].head(365))
            longPoints_20_100 = FindTheBuyPoint(self.SMA['SMA20'].head(365), self.SMA['SMA100'].head(365))
            shortPoints_100_5 = FindTheSellPoint(self.SMA['SMA20'].head(365), self.SMA['SMA100'].head(365))
        elif(indicator.lower() == 'ema'):
            print ("123")
        elif(indicator.lower() == 'rsi'):
            print ("123")
        elif(indicator.lower() == 'kd'):
            longPoints = FindTheBuyPoint(self.KD['K9'].head(365), self.KD['D9'].head(365))
            shortPoints = FindTheSellPoint(self.KD['K9'].head(365), self.KD['D9'].head(365))
        elif(indicator.lower() == 'macd'):
            print ("123")
        else:
            print ("Error: the content of argument is illegal")
            return
        return longPoints, shortPoints

In [6]:
#get the cryptocurrency price from DB

try:
    sql = "SELECT LAST, DATE, HIGH, LOW FROM btc_usd ORDER BY DATE"
    results = GetDataFromDB("140.118.126.136", "123", "1234567890", "test", sql)
    Dates = []
    ClosePrices = []
    HighPrices = []
    LowPrices = []
    for i in range(len(results)):
        Dates.append(results[i][1])
        ClosePrices.append(results[i][0])
        HighPrices.append(results[i][2])
        LowPrices.append(results[i][3])
        
    CoinPriceData = {
        'Date' : Dates,
        'Close' : ClosePrices,
        'High' : HighPrices,
        'Low' : LowPrices
    }

    CoinPriceData = pd.DataFrame(CoinPriceData)
except:
    print ("Error: unable to fetch data from DB")

In [84]:
sma5 = SMA(CoinPriceData['Close'], 5)
sma5 = sma5.fillna(0)
sma10 = SMA(CoinPriceData['Close'], 10)
sma10 = sma10.fillna(0)
sma20 = SMA(CoinPriceData['Close'], 20)
sma20 = sma20.fillna(0)
sma60 = SMA(CoinPriceData['Close'], 60)
sma60 = sma60.fillna(0)
sma120 = SMA(CoinPriceData['Close'], 120)
sma120 = sma120.fillna(0)
sma240 = SMA(CoinPriceData['Close'], 240)
sma240 = sma240.fillna(0)
rsi, up, down = RSI(CoinPriceData['Close'], 14)
rsi = rsi.fillna(0)
up = up.fillna(0)
down = down.fillna(0)
kdresult = KD(CoinPriceData)
k = kdresult['K9']
d = kdresult['D9']
macd = MACD(CoinPriceData, [12, 26, 9])
dif = macd['dif']
dem = macd['dem']
sql = "INSERT INTO indicators (DATE, SMA240, SMA120, SMA60, SMA20, SMA10, SMA5, RSI, K, D, DIF, DEM, UP, DOWN) VALUES "
for i in range(len(CoinPriceData['Date'])):
    if i != 0:
        sql += ","
    sql += "("
    sql += "'" + str(CoinPriceData['Date'][i]) + "'" + ","
    sql += str(sma240[i]) + ","
    sql += str(sma120[i]) + ","
    sql += str(sma60[i]) + ","
    sql += str(sma20[i]) + ","
    sql += str(sma10[i]) + ","
    sql += str(sma5[i]) + ","
    sql += str(rsi[i]) + ","
    sql += str(k[i]) + ","
    sql += str(d[i]) + ","
    sql += str(dif[i]) + ","
    sql += str(dem[i]) + ","
    sql += str(up[i]) + ","
    sql += str(down[i])
    sql += ")"
sql += ";"

In [85]:
print(sql)
db = pymysql.connect("140.118.126.136", "123", "1234567890", "test")
cursor = db.cursor()
cursor.execute(sql)
db.close()

INSERT INTO indicators (DATE, SMA240, SMA120, SMA60, SMA20, SMA10, SMA5, RSI, K, D, DIF, DEM, UP, DOWN) VALUES ('2014-04-15 00:00:00',0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0),('2014-04-16 00:00:00',0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0),('2014-04-17 00:00:00',0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0),('2014-04-18 00:00:00',0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0),('2014-04-19 00:00:00',0.0,0.0,0.0,0.0,0.0,508.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0),('2014-04-20 00:00:00',0.0,0.0,0.0,0.0,0.0,507.538,0.0,0.0,0.0,0.0,0.0,0.0,0.0),('2014-04-21 00:00:00',0.0,0.0,0.0,0.0,0.0,499.338,0.0,0.0,0.0,0.0,0.0,0.0,0.0),('2014-04-22 00:00:00',0.0,0.0,0.0,0.0,0.0,495.97799999999995,0.0,0.0,0.0,0.0,0.0,0.0,0.0),('2014-04-23 00:00:00',0.0,0.0,0.0,0.0,0.0,497.434,0.0,0.13344,0.04448,0.0,0.0,0.0,0.0),('2014-04-24 00:00:00',0.0,0.0,0.0,0.0,501.89,495.53000000000003,0.0,0.19898,0.09598,0.0,0.0,0.0,0.0),('2014-04-25 00:00:00',0.0,0.0,0.0,0.0,497.76899999999995,488.0,0.0,0

In [87]:
a = GetDataFromDB("140.118.126.136", "123", "1234567890", "test", "SELECT * FROM test.indicators")
print(a)

(('2014-04-15 00:00:00', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), ('2014-04-16 00:00:00', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), ('2014-04-17 00:00:00', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), ('2014-04-18 00:00:00', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), ('2014-04-19 00:00:00', 0.0, 0.0, 0.0, 0.0, 0.0, 508.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), ('2014-04-20 00:00:00', 0.0, 0.0, 0.0, 0.0, 0.0, 507.538, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), ('2014-04-21 00:00:00', 0.0, 0.0, 0.0, 0.0, 0.0, 499.338, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), ('2014-04-22 00:00:00', 0.0, 0.0, 0.0, 0.0, 0.0, 495.978, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), ('2014-04-23 00:00:00', 0.0, 0.0, 0.0, 0.0, 0.0, 497.434, 0.0, 0.13344, 0.04448, 0.0, 0.0, 0.0, 0.0), ('2014-04-24 00:00:00', 0.0, 0.0, 0.0, 0.0, 501.89, 495.53, 0.0, 0.19898, 0.09598, 0.0, 0.0, 0.0, 0.0), ('2014-04-25 00:00:00', 0.0, 0.0, 0.0, 0.0, 497.769, 488.0

In [7]:
rsiR('2018/03/12')
kdR('2018/03/12')

UPDATE INDICATORS SET RSI = 0.0 WHERE DATE = 2018/03/12


IndexError: tuple index out of range

In [18]:
#define a class variable with class Indicators
indicators = Indicators(CoinPriceData)

In [19]:
# get long, short points by all indicators
##LPoint_sma, SPoint_sma = indicators.getLong_ShortPoints('sma')
LPoint_sma, SPoint_ema = indicators.getLong_ShortPoints('ema')
LPoint_sma, SPoint_kd = indicators.getLong_ShortPoints('kd')
LPoint_sma, SPoint_rsi = indicators.getLong_ShortPoints('rsi')
LPoint_sma, SPoint_macd = indicators.getLong_ShortPoints('macd')

123
123
123
